In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import os

from utils.dataloader import *
from utils.utils import *
from utils.preprocessing import traj_preprocessing

# Challenge Dataset unzip

In [2]:
file_name = "public_data_challenge_v0.zip"
output_dir = "./challenge_dataset"
if not os.path.exists("./challenge_dataset/track_1/exp_1/videos_fov_10.tiff"):
    os.system("unzip "+file_name+" -d "+output_dir)

In [10]:
PATH = "./challenge_dataset/track_2"

In [13]:
results_path = "challenge_results"

# Load Trained Model

In [14]:
from utils.resnet import resnet18_andi, resnet34_andi

dataset_size = 150

predictors = {f"{feat}":[]
              for feat in ["a", "k", "s"]
              }

for d, feat in zip([1,1,4,1], ["a", "k", "s"]):
    model = resnet34_andi(output_dim=d, output_num=1, problem_type=feat)
    model.load_state_dict(torch.load(f"./model/predictor_{feat}_full_{dataset_size}/{feat}_ckpt_best.pt"))
    model.eval()
    predictors[f"{feat}"] = model
          
init_predictor = resnet18_andi(output_dim=1, output_num=1, problem_type="cp")
init_predictor.load_state_dict(torch.load(f"./model/predictor_cp_step_full_200/cp_ckpt_best.pt"))
init_predictor.eval()

cp_classifier = resnet34_andi(output_dim=2, output_num=1, problem_type="s")
cp_classifier.load_state_dict(torch.load(f"./model/classifier_cp_full_{dataset_size}/cp_ckpt_best.pt"))
cp_classifier.eval()
"Done"
####################

In [ ]:
# 

In [16]:
for exp_dir in tqdm(os.listdir(PATH)):
    print(exp_dir)
    for fov in tqdm(range(30)):
        val_dir = f"./{results_path}/res/track_2/{exp_dir}/"
        if not os.path.exists(val_dir):
            createFolder(val_dir)
        
        submission_file = val_dir + f"fov_{fov}.txt"
        
        traj_df = pd.read_csv(PATH + f"/{exp_dir}/" + f"trajs_fov_{fov}.csv")
        traj_groups = traj_df.groupby('traj_idx').apply(lambda g: g[['x', 'y']].to_numpy().T, include_groups=False)
        raw_traj_list = list(traj_groups)

        trajs_list = []
        trajs_raw_length = []
        for raw_traj in raw_traj_list:
            raw_length = len(raw_traj[0])
            traj = traj_preprocessing(raw_traj)
        
            trajs_raw_length.append(raw_length)
            trajs_list.append(traj)

        with open(submission_file, 'w') as f:   
            for traj_index, trajs in (enumerate(trajs_list)):
                init_cp = 0
                prediction_traj = [traj_index]
                current_traj = trajs
                raw_length = trajs_raw_length[traj_index]
                prev_init_cp = 0
                true_init_cp = 0
                prev_pad_len = 200 - raw_length

                cp_existence = True
                while True:
                    # cp existence check
                    trajs_tensor = torch.Tensor(np.array(current_traj)).unsqueeze(axis=0).unsqueeze(axis=2)
                    
                    cp_existence = cp_classifier(trajs_tensor)
                    cp_existence = torch.argmax(cp_existence).item()
                    
                    # padding consideration, finding cp with predict step size
                    
                    cp_step = init_predictor(trajs_tensor)
                    cp_step = round(cp_step.detach().cpu().item())
                    cp_pos = prev_pad_len + cp_step  # cp position in padded trajs (len 200)
                    raw_cp_pos = int(cp_pos - 200 + raw_length) # cp position in raw trajs (len raw length)
                    
                    # print(raw_cp_pos, cp_step, cp_pos, raw_length)
                    if cp_existence==False:
                        raw_cp_pos = raw_length
                    if cp_step==0:
                        print(cp_step, cp_pos, raw_length)
                    # min(segement) = 3, make over 197 as just 200
                    if cp_pos >= 197:
                        raw_cp_pos = raw_length
                        cp_pos = 200
                    # current trajectory feature extraction
                    for feature in [ "k", "a", "s"]:
                        feat_predictor = predictors[feature]
                        feat = feat_predictor(trajs_tensor)
                        if feature == "s":
                            prediction_traj.append(torch.argmax(feat).item())
                        else:
                            prediction_traj.append(feat.item())
                    
                    prediction_traj.append(raw_cp_pos)

                    # next step preparation. trajectory cutting & current cp position save
                    current_traj = current_traj[:, cp_pos:]
                    prev_pad_len = cp_pos
                    # condition, break loop
                    # 1. cp pos is 200
                    if current_traj.shape[-1] == 0:
                        break
                    # 2. cp_existence condition
                    if raw_cp_pos == raw_length:
                        break
                        
                    current_traj = traj_preprocessing(current_traj)


                # print(prediction_traj)
                formatted_numbers = ','.join(map(str, prediction_traj))
                f.write(formatted_numbers + '\n')


  0%|          | 0/12 [00:00<?, ?it/s]

exp_7



  8%|▊         | 1/12 [16:58<3:06:42, 1018.42s/it]

exp_5



 17%|█▋        | 2/12 [17:48<1:14:50, 449.09s/it] 

...


 83%|████████▎ | 10/12 [34:56<07:06, 213.29s/it]

exp_2



 92%|█████████▏| 11/12 [36:06<02:49, 169.57s/it]

exp_9



100%|██████████| 12/12 [37:54<00:00, 189.55s/it]


In [17]:
import shutil
shutil.make_archive(f"./{results_path}/res", 'zip', f"./{results_path}/res")

'/home/bjyong/andi2/challenge_results_model_0715/res.zip'

In [18]:
# ensemble

In [19]:
from utils.ensemble import *

In [20]:
base_dir = f'./{results_path}/res/track_2'

In [21]:
for dirs in tqdm(os.listdir(base_dir)):
    exp_dir = base_dir + f"/{dirs}/"
    # Create the folder of the experiment if it does not exits

    file_name = exp_dir + 'ensemble_labels.txt'
    # threshold ~ 3\sigma
    results = get_ensemble_results(exp_dir, max_iter=2500, threshold=0.03/100, verbose=False)
    n_state = results.shape[-1]
    print(results)
    with open(file_name, 'w') as f:
        # Save the model (random) and the number of states (2 in this case)
        model_name = "multi-state"
        
        f.write(f'model: {model_name}; num_state: {n_state} \n')
    
        # Create some dummy data for 2 states. This means 2 columns
        # and 5 rows
        
        # Save the data in the corresponding ensemble file
        np.savetxt(f, results, delimiter = ';')

  8%|▊         | 1/12 [02:24<26:28, 144.37s/it]

[[1.11600089e+00 8.94380562e-01 4.89238130e-01 1.48120850e+00]
 [1.27682792e-01 2.93224954e-02 3.85906832e-03 1.84494854e-03]
 [8.23600996e-02 9.71048935e-01 5.66160889e-01 1.09781666e+00]
 [9.70752158e-04 1.15535669e-03 8.69568802e-02 1.26513747e-03]
 [2.60897203e-01 6.11826462e-01 8.28669363e-02 4.44093985e-02]]



 17%|█▋        | 2/12 [02:26<10:08, 60.85s/it] 

[[1.01711343e+00 1.02798700e+00]
 [1.75689031e-03 2.61163011e-02]
 [9.56870868e-01 3.77775396e-01]
 [4.20730372e-04 1.06344304e-01]
 [8.62202277e-01 1.37797723e-01]]



 25%|██▌       | 3/12 [02:34<05:27, 36.38s/it]

...

[[1.09723611e+00 9.87633623e-01 6.29295361e-01]
 [1.68180449e-03 3.70618983e-03 1.52198744e-01]
 [5.81651335e-01 3.60676987e-02 9.72213872e-01]
 [9.81199804e-02 1.02949830e-03 1.05198040e-03]
 [4.81462675e-02 3.14770421e-01 6.37083312e-01]]



 92%|█████████▏| 11/12 [04:31<00:16, 16.46s/it]

[[1.00947747e+00 1.01805567e+00 8.03885532e-02]
 [9.52952988e-04 1.71172172e-02 2.31271720e-02]
 [1.23183983e-05 9.64220781e-01 2.32387825e-01]
 [1.03746161e-06 1.26219093e-03 7.95435166e-02]
 [1.20194566e-01 7.89384313e-01 9.04211210e-02]]



100%|██████████| 12/12 [04:38<00:00, 23.23s/it]

[[0.93664972 0.971549   0.47448737]
 [0.01947819 0.17616113 0.00734077]
 [0.96441868 0.09570621 0.48434085]
 [0.00174773 0.00244202 0.08071059]
 [0.46199524 0.40836298 0.12964179]]


In [22]:
"Done"

'Done'

In [23]:
import shutil
shutil.make_archive(f"./{results_path}/res", 'zip', f"./{results_path}/res")

'/home/bjyong/andi2/challenge_results_model_0715/res.zip'

In [24]:
import os

def find_all_files(root_dir):
    """특정 디렉토리 이하의 모든 파일 경로를 찾습니다."""
    file_paths = []
    for dirpath, _, filenames in os.walk(root_dir):
        for name in filenames:
            if name.endswith('.txt'):
                file_paths.append(os.path.relpath(os.path.join(dirpath, name), root_dir))
    return file_paths

def read_file_lines(file_path):
    """파일 내용을 한 줄씩 읽습니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

def compare_files(root_dir1, root_dir2):
    """두 루트 디렉토리 하위의 모든 파일을 비교하여 서로 다른 라인을 출력합니다."""
    files1 = find_all_files(root_dir1)
    files2 = find_all_files(root_dir2)
    
    common_files = set(files1).intersection(files2)
    if not common_files:
        print("두 디렉토리 내에 공통된 파일이 없습니다.")
        return

    for file in common_files:
        path1 = os.path.join(root_dir1, file)
        path2 = os.path.join(root_dir2, file)
        
        lines1 = read_file_lines(path1)
        lines2 = read_file_lines(path2)
        
        max_length = max(len(lines1), len(lines2))
        
        print(f"\nComparing {path1} and {path2}:")
        for i in range(max_length):
            line1 = lines1[i] if i < len(lines1) else ''
            line2 = lines2[i] if i < len(lines2) else ''
            if line1 != line2:
                print(f"Line {i + 1}:")
                print(f"- {path1}: {line1.strip()}")
                print(f"- {path2}: {line2.strip()}")

def main(root_dir1, root_dir2):
    compare_files(root_dir1, root_dir2)



In [25]:
# 사용 예시
root_directory1 = './challenge_results_model_0715/res'  # 첫 번째 루트 디렉토리를 지정하세요.
root_directory2 = './challenge_results_model_0710/res'  # 두 번째 루트 디렉토리를 지정하세요.
main(root_directory1, root_directory2)



Comparing ./challenge_results_model_0715/res/track_2/exp_10/fov_10.txt and ./challenge_results_model_0710/res/track_2/exp_10/fov_10.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_4/fov_29.txt and ./challenge_results_model_0710/res/track_2/exp_4/fov_29.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_7/fov_24.txt and ./challenge_results_model_0710/res/track_2/exp_7/fov_24.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_7/fov_23.txt and ./challenge_results_model_0710/res/track_2/exp_7/fov_23.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_4/fov_7.txt and ./challenge_results_model_0710/res/track_2/exp_4/fov_7.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_5/fov_25.txt and ./challenge_results_model_0710/res/track_2/exp_5/fov_25.txt:

Comparing ./challenge_results_model_0715/res/track_2/exp_5/fov_0.txt and ./challenge_results_model_0710/res/track_2/exp_5/fov_0.txt:

Comparing ./challenge_results_model_0715/res/trac